In [1]:
import os
print("CWD antes:", os.getcwd())  # dónde está apuntando el notebook

# Si esto imprime .../project_esports_occidente/notebooks
# entonces subimos a la carpeta raíz del proyecto:
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

print("CWD ahora:", os.getcwd())  # debería terminar en .../project_esports_occidente


CWD antes: c:\Users\cubea\OneDrive\Escritorio\project_esports_occidente\notebooks
CWD ahora: c:\Users\cubea\OneDrive\Escritorio\project_esports_occidente


In [2]:
import pandas as pd

twitch_path   = "data/raw/top_twitch/Twitch_game_data.csv"
earnings_path = "data/raw/esports_earnings/HistoricalEsportData.csv"

def safe_read_csv(path):
    # 1) intenta utf-8 con separador auto
    try:
        return pd.read_csv(path)
    except UnicodeDecodeError:
        pass
    # 2) intenta cp1252/latin-1 (típico Windows)
    for enc in ("cp1252", "latin1", "utf-16"):
        try:
            return pd.read_csv(path, encoding=enc)
        except UnicodeDecodeError:
            continue
        except Exception:
            continue
    # 3) último recurso: encoding flexible y separador ';'
    return pd.read_csv(path, encoding="cp1252", sep=";", engine="python", on_bad_lines="skip", errors="ignore")

twitch_df   = safe_read_csv(twitch_path)
earnings_df = safe_read_csv(earnings_path)

display(twitch_df.head())
display(earnings_df.head())
twitch_df.info()
earnings_df.info()



,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
0,1,League of Legends,1,2016,94377226,1362044,530270,2903,129172,127021,1833,69.29
1,2,Counter-Strike: Global Offensive,1,2016,47832863,830105,372654,2197,120849,64378,1117,57.62
2,3,Dota 2,1,2016,45185893,433397,315083,1100,44074,60815,583,104.26
3,4,Hearthstone,1,2016,39936159,235903,131357,517,36170,53749,317,169.29
4,5,Call of Duty: Black Ops III,1,2016,16153057,1151578,71639,3620,214054,21740,1549,14.03


,Date,Game,Earnings,Players,Tournaments
0,1998-01-01,Command & Conquer: Red Alert,15000.0,8,1
1,1998-01-01,QuakeWorld,15000.0,8,1
2,1998-05-01,Quake II,15000.0,4,1
3,1998-05-01,Total Annihilation,15000.0,2,1
4,1998-05-01,QuakeWorld,300.0,3,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Rank              21000 non-null  int64  
 1   Game              20999 non-null  object 
 2   Month             21000 non-null  int64  
 3   Year              21000 non-null  int64  
 4   Hours_watched     21000 non-null  int64  
 5   Hours_streamed    21000 non-null  int64  
 6   Peak_viewers      21000 non-null  int64  
 7   Peak_channels     21000 non-null  int64  
 8   Streamers         21000 non-null  int64  
 9   Avg_viewers       21000 non-null  int64  
 10  Avg_channels      21000 non-null  int64  
 11  Avg_viewer_ratio  21000 non-null  float64
dtypes: float64(1), int64(10), object(1)
memory usage: 1.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10239 entries, 0 to 10238
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --

In [3]:
t = twitch_df.rename(columns={
    "Game":"juego","Year":"año","Month":"mes",
    "Avg_viewers":"media_viewers","Peak_viewers":"pico_viewers","Hours_watched":"horas_vistas"
})[["juego","año","mes","media_viewers","pico_viewers","horas_vistas"]]


In [4]:
def nrm(s):
    m={"cs:go":"csgo","counter-strike: global offensive":"csgo","counter-strike 2":"cs2",
       "league of legends":"lol","dota 2":"dota2","pubg: battlegrounds":"pubg",
       "overwatch 2":"overwatch2","ea sports fc 24":"ea fc"}
    s=str(s).strip().lower()
    return m.get(s,s)
t["juego_norm"]=t["juego"].map(nrm)


In [5]:
import pandas as pd

# ruta correcta del dataset
hist_path = "data/raw/esports_earnings/HistoricalEsportData.csv"

# carga segura (prueba separador y encoding si fuera necesario)
hist_df = pd.read_csv(hist_path, encoding="cp1252", sep=",")

# vistazo rápido opcional
hist_df.head()



,Date,Game,Earnings,Players,Tournaments
0,1998-01-01,Command & Conquer: Red Alert,15000.0,8,1
1,1998-01-01,QuakeWorld,15000.0,8,1
2,1998-05-01,Quake II,15000.0,4,1
3,1998-05-01,Total Annihilation,15000.0,2,1
4,1998-05-01,QuakeWorld,300.0,3,1


In [6]:
e = hist_df.rename(columns={"Game":"juego","Earnings":"premios_usd","Tournaments":"torneos"})[
    ["juego","premios_usd","torneos","Date"]
]
e["año"] = pd.to_datetime(e["Date"], errors="coerce").dt.year
e["juego_norm"] = e["juego"].map(nrm)
e_ag = e.groupby(["juego_norm","año"], dropna=False)[["premios_usd","torneos"]].sum().reset_index()


In [8]:
t_ag = (t.groupby(["juego_norm","año"], dropna=False)
          .agg(media_viewers=("media_viewers","mean"),
               pico_viewers=("pico_viewers","max"),
               horas_vistas=("horas_vistas","sum"))
          .reset_index())

t_ag.head()


,juego_norm,año,media_viewers,pico_viewers,horas_vistas
0,.hack//g.u. last recode,2017,202.000000,1222,145350
1,20 minutes till dawn,2022,1267.000000,29743,911356
2,2xko,2024,2876.000000,60961,2137305
3,60 parsecs!,2018,736.000000,31960,529688
4,60 seconds!,2016,517.666667,56904,1151013


In [9]:
df_merge = t_ag.merge(e_ag, on=["juego_norm","año"], how="left")
df_merge.head()

,juego_norm,año,media_viewers,pico_viewers,horas_vistas,premios_usd,torneos
0,.hack//g.u. last recode,2017,202.000000,1222,145350,NaN,NaN
1,20 minutes till dawn,2022,1267.000000,29743,911356,NaN,NaN
2,2xko,2024,2876.000000,60961,2137305,NaN,NaN
3,60 parsecs!,2018,736.000000,31960,529688,NaN,NaN
4,60 seconds!,2016,517.666667,56904,1151013,NaN,NaN


In [10]:
g_path = "data/raw/esports_earnings/GeneralEsportData.csv"
g = pd.read_csv(g_path, encoding="cp1252")

g = g.rename(columns={
    "Game":"juego",
    "Genre":"genero",
    "TotalEarnings":"premios_totales_usd",
    "TotalPlayers":"jugadores_totales",
    "TotalTournaments":"torneos_totales"
})[["juego","genero","premios_totales_usd","jugadores_totales","torneos_totales"]]

g["juego_norm"] = g["juego"].map(nrm)
g.head()


,juego,genero,premios_totales_usd,jugadores_totales,torneos_totales,juego_norm
0,Age of Empires,Strategy,736284.75,624,341,age of empires
1,Age of Empires II,Strategy,3898508.73,2256,1939,age of empires ii
2,Age of Empires III,Strategy,122256.72,172,179,age of empires iii
3,Age of Empires IV,Strategy,1190813.44,643,423,age of empires iv
4,Age of Empires Online,Strategy,11462.98,52,68,age of empires online


In [11]:
df_final = df_merge.merge(
    g[["juego_norm","genero","premios_totales_usd","jugadores_totales","torneos_totales"]],
    on="juego_norm", how="left"
)
df_final.head()


,juego_norm,año,media_viewers,pico_viewers,horas_vistas,premios_usd,torneos,genero,premios_totales_usd,jugadores_totales,torneos_totales
0,.hack//g.u. last recode,2017,202.000000,1222,145350,NaN,NaN,NaN,NaN,NaN,NaN
1,20 minutes till dawn,2022,1267.000000,29743,911356,NaN,NaN,NaN,NaN,NaN,NaN
2,2xko,2024,2876.000000,60961,2137305,NaN,NaN,NaN,NaN,NaN,NaN
3,60 parsecs!,2018,736.000000,31960,529688,NaN,NaN,NaN,NaN,NaN,NaN
4,60 seconds!,2016,517.666667,56904,1151013,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_final.info()
df_final.isna().sum().sort_values(ascending=False).head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4700 entries, 0 to 4699
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   juego_norm           4700 non-null   object 
 1   año                  4700 non-null   int64  
 2   media_viewers        4700 non-null   float64
 3   pico_viewers         4700 non-null   int64  
 4   horas_vistas         4700 non-null   int64  
 5   premios_usd          521 non-null    float64
 6   torneos              521 non-null    float64
 7   genero               769 non-null    object 
 8   premios_totales_usd  769 non-null    float64
 9   jugadores_totales    769 non-null    float64
 10  torneos_totales      769 non-null    float64
dtypes: float64(6), int64(3), object(2)
memory usage: 404.0+ KB


torneos                4179
premios_usd            4179
premios_totales_usd    3931
jugadores_totales      3931
torneos_totales        3931
genero                 3931
juego_norm                0
media_viewers             0
año                       0
pico_viewers              0
dtype: int64

In [13]:
from pathlib import Path
PRO = Path("data/processed"); PRO.mkdir(parents=True, exist_ok=True)
df_final.to_csv(PRO/"esports_occidente_final.csv", index=False)
print("✅ Dataset final guardado en data/processed/esports_occidente_final.csv")


✅ Dataset final guardado en data/processed/esports_occidente_final.csv


In [14]:
# Twitch agregado
t_ag = t_ag.rename(columns={
    "juego_norm":"JuegoNorm", "año":"Año",
    "media_viewers":"MediaViewers", "pico_viewers":"PicoViewers",
    "horas_vistas":"HorasVistas"
})

# Esports agregado
e_ag = e_ag.rename(columns={
    "juego_norm":"JuegoNorm", "año":"Año",
    "premios_usd":"PremiosUSD", "torneos":"Torneos"
})

# General (dimensión juego)
g = g.rename(columns={
    "juego_norm":"JuegoNorm",
    "genero":"Género",
    "premios_totales_usd":"PremiosTotalesUSD",
    "jugadores_totales":"JugadoresTotales",
    "torneos_totales":"TorneosTotales"
})


In [15]:
df_merge = t_ag.merge(e_ag, on=["JuegoNorm","Año"], how="left")
df_final = df_merge.merge(
    g[["JuegoNorm","Género","PremiosTotalesUSD","JugadoresTotales","TorneosTotales"]],
    on="JuegoNorm", how="left"
)


In [16]:
cols = ["JuegoNorm","Año","MediaViewers","PicoViewers","HorasVistas",
        "PremiosUSD","Torneos","Género","PremiosTotalesUSD","JugadoresTotales","TorneosTotales"]
df_final = df_final.reindex(columns=cols)
df_final.to_csv("data/processed/Esports_Occidente_Final.csv", index=False)


In [17]:
# 1) Asegura tipos
e_ag["Año"] = pd.to_numeric(e_ag["Año"], errors="coerce")

# 2) Filtra premios 2016–2024
e_1624 = e_ag.query("2016 <= Año <= 2024").copy()

# 3) (Opcional) re-agrega por JuegoNorm-Año por si hubiera duplicados
e_1624 = (e_1624
          .groupby(["JuegoNorm","Año"], as_index=False)[["PremiosUSD","Torneos"]]
          .sum())

# 4) (Opcional) si ya tienes df_merge/df_final y quieres limitar el análisis a ese rango:
df_final_1624 = df_final[df_final["Año"].between(2016, 2024)].copy()

# 5) Guardar
e_1624.to_csv("data/processed/Tournaments_2016_2024.csv", index=False)
df_final_1624.to_csv("data/processed/Esports_Occidente_2016_2024.csv", index=False)


In [18]:
# sanity check
e_ag["Año"].min(), e_ag["Año"].max()   # para confirmar cobertura real


(1998, 2024)

In [19]:
df_final_1624 = df_final[df_final["Año"].between(2016, 2024)].copy()
df_final_1624.drop_duplicates(inplace=True)
df_final_1624.isna().sum().sort_values(ascending=False).head(10)



Torneos              4179
PremiosUSD           4179
PremiosTotalesUSD    3931
JugadoresTotales     3931
TorneosTotales       3931
Género               3931
JuegoNorm               0
MediaViewers            0
Año                     0
PicoViewers             0
dtype: int64

In [20]:
kpis = (df_final_1624
        .groupby("JuegoNorm", as_index=False)
        .agg(MediaViewers=("MediaViewers","mean"),
             PicoViewers=("PicoViewers","max"),
             HorasVistas=("HorasVistas","sum"),
             PremiosUSD=("PremiosUSD","sum"),
             Torneos=("Torneos","sum")))


In [21]:
top_viewers = kpis.sort_values("MediaViewers", ascending=False).head(10)
top_premios = kpis.sort_values("PremiosUSD", ascending=False).head(10)
top_viewers, top_premios


(                  JuegoNorm   MediaViewers  PicoViewers  HorasVistas  \
 959           just chatting  251760.793651      2787896  14397469625   
 1060                    lol  147528.206790      3145012  11296781271   
 2195               valorant  117624.566667      1728977   4631862775   
 783      grand theft auto v   99826.030864      1140656   7482026514   
 701                fortnite   92210.135417      3291525   6224642826   
 327          counter-strike   74317.817460      1548040    876395705   
 356                    csgo   68367.289583      1916027   4315553263   
 492                   dota2   58927.419753      1636441   4503767707   
 257   call of duty: warzone   56547.463889      1629284   2764017645   
 74             apex legends   52917.957492       666868   2655211457   
 
         PremiosUSD  Torneos  
 959   0.000000e+00      0.0  
 1060  8.667619e+07   1297.0  
 2195  3.201822e+07   1316.0  
 783   0.000000e+00      0.0  
 701   1.910127e+08   2293.0  
 327   0.

In [22]:
scatter_df = kpis[["JuegoNorm","MediaViewers","PremiosUSD"]].dropna()
scatter_df.head()


,JuegoNorm,MediaViewers,PremiosUSD
0,.hack//g.u. last recode,202.000000,0.0
1,20 minutes till dawn,1267.000000,0.0
2,2xko,2876.000000,0.0
3,60 parsecs!,736.000000,0.0
4,60 seconds!,415.416667,0.0


In [23]:
from pathlib import Path
Path("data/processed").mkdir(parents=True, exist_ok=True)
kpis.to_csv("data/processed/KPIs_2016_2024.csv", index=False)
top_viewers.to_csv("data/processed/Top_Viewers_2016_2024.csv", index=False)
top_premios.to_csv("data/processed/Top_Premios_2016_2024.csv", index=False)
scatter_df.to_csv("data/processed/Scatter_Viewers_Premios_2016_2024.csv", index=False)


In [24]:
df_final_1624.groupby("Año").size()


Año
2016    619
2017    597
2018    586
2019    522
2020    502
2021    488
2022    467
2023    508
2024    411
dtype: int64